### This script is to:
* combine parcel10 with PLU10 (data used in PBA40 as the "base truth")
* combine parcel10 with BASIS PLU (hybrid data for Blueprint Option 2)
* compare the build-out capacity under different PLU sources (BASIS PLU data validation)
* combine parcel10 with PBA50 zoningmodcats (hybrid data for Blueprint Option 2)
* compare the development capacity under different zoningmodcat

In [292]:
import pandas as pd
import numpy as np

In [403]:
cnty = {
'Alameda': 1.0,
'Contra Costa': 13.0,
'Marin': 41.0,
'Napa': 55.0,
'San Francisco': 75.0,
'San Mateo': 81.0,
'Santa Clara': 85.0,
'Solano': 95.0,
'Sonoma': 97.0}

ctyMap = pd.DataFrame(cnty.items(), columns=['ctyName', 'ctyCode'])
#ctyMap['ctyCode'] = ctyMap['ctyCode'].apply(lambda x: int(x))
#ctyMap

## PBA40 zoning code PLU

In [341]:
plu10 = pd.read_csv('C:/Users/ywang/Documents/GitHub/bayarea_urbansim/data/zoning_lookup.csv')

In [342]:
plu10['jz_o'] = plu10['city'].str.cat(plu10['name'],sep=" ")

In [343]:
print(plu10.shape[0], len(plu10.id.unique()), len(plu10.jz_o.unique()))

5156 5156 4536


In [344]:
plu10.head()

,id,juris,city,name,max_far,max_height,max_dua,max_du_per_parcel,HS,HT,...,IL,IW,IH,RS,RB,MR,MT,ME,plandate,jz_o
0,2101,1.0,Albany,RHD,0.50,35.0,9.0,NaN,1,0,...,0,0,0,0,0,0,0,0,NaN,Albany RHD
1,2102,1.0,Albany,R-1,0.55,28.0,12.0,NaN,1,0,...,0,0,0,0,0,0,0,0,NaN,Albany R-1
2,2103,1.0,Albany,R-2,0.55,35.0,35.0,NaN,1,1,...,0,0,0,0,0,0,0,0,NaN,Albany R-2
3,2104,1.0,Albany,R-3,1.50,35.0,63.0,NaN,1,1,...,0,0,0,0,0,0,0,0,NaN,Albany R-3
4,2105,1.0,Albany,R.4,NaN,NaN,87.0,NaN,0,0,...,0,0,0,0,0,0,0,0,NaN,Albany R.4


In [345]:
plu10['id'] = plu10['id'].apply(lambda x: float(x))

In [346]:
plu10.dtypes

id                   float64
juris                float64
city                  object
name                  object
max_far              float64
max_height           float64
max_dua              float64
max_du_per_parcel    float64
HS                     int64
HT                     int64
HM                     int64
OF                     int64
HO                     int64
SC                    object
IL                     int64
IW                     int64
IH                     int64
RS                     int64
RB                     int64
MR                     int64
MT                     int64
ME                     int64
plandate              object
jz_o                  object
dtype: object

## Parcel 10 geometry and Zoning Code

In [347]:
# parcel geometry
pcl10 = pd.read_csv('C:/Users/ywang/Box/Mine/3_BASIS/pcl10.csv',usecols=['PARCEL_ID','APN','geom_id_s','COUNTY_ID',
                                                                         'manual_county','jurisdiction',
                                                                         'LAND_VALUE','ACRES','TAX_EXEMPT',
                                                                         'pda_id','tpp_id','exp_id','exp_score','opp_id',
                                                                         'zoningmodcat','perffoot','perfarea','mapshell',
                                                                         'tpa_id','perfarea2','alt_zoning','zonetype'])
pcl10.shape

(1956208, 22)

In [348]:
pcl10.head()

,PARCEL_ID,LAND_VALUE,ACRES,COUNTY_ID,TAX_EXEMPT,APN,geom_id_s,manual_county,jurisdiction,pda_id,...,exp_score,opp_id,zoningmodcat,perffoot,perfarea,mapshell,tpa_id,perfarea2,alt_zoning,zonetype
0,229116.0,0.0,3.360520,1.0,0.0,099 029001700,10305106092872,1,41992,NaN,...,0.0,NaN,41992NANANANA,1,0,1.0,NaN,0,NaN,NaN
1,244166.0,0.0,1.294423,1.0,0.0,099B540210200,11107351665227,1,41992,NaN,...,0.0,NaN,41992NANANANA,1,0,1.0,NaN,0,NaN,NaN
2,202378.0,6036500.0,14.993605,1.0,1.0,085A643106000,11030175960628,1,33000,NaN,...,0.0,NaN,33000NANANANA,1,0,1.0,NaN,0,NaN,NaN
3,2004420.0,179954.0,316.247146,97.0,0.0,141-100-012,6381677629073,97,97,NaN,...,0.0,NaN,00097NANANANA,0,0,NaN,NaN,0,NaN,NaN
4,340332.0,0.0,0.621275,1.0,0.0,525 166004800,314875459798,1,26000,NaN,...,0.0,NaN,26000NAb1NANA,1,1,1.0,bus,1,TE,TPA


In [349]:
# pacel to zoning code mapping
pz10 = pd.read_csv('C:/Users/ywang/Box/Mine/1_UrbanSim/Current Large General Input Data/2015_12_21_zoning_parcels.csv')
pz10.shape

(1950733, 8)

In [350]:
pz10.head()

,geom_id,Unnamed: 0,zoning_id,zoning,juris,prop,tablename,nodev
0,1846247885201,0,12202.0,115 - Residential 0-4 du/ac,102,100,alamedacountygp2006db,0
1,11768793521677,1,12204.0,115 - Residential 9-17 du/ac,102,100,alamedacountygp2006db,0
2,807545210880,2,12204.0,115 - Residential 9-17 du/ac,102,100,alamedacountygp2006db,0
3,8785012057974,3,12204.0,115 - Residential 9-17 du/ac,102,100,alamedacountygp2006db,0
4,14057552282712,4,12204.0,115 - Residential 9-17 du/ac,102,100,alamedacountygp2006db,0


In [351]:
# Merge
p10_z10 = pcl10.merge(pz10, left_on = 'geom_id_s', right_on = 'geom_id', how = 'left')

In [352]:
print(p10_z10.shape)
p10_z10.head()

(1956208, 30)


,PARCEL_ID,LAND_VALUE,ACRES,COUNTY_ID,TAX_EXEMPT,APN,geom_id_s,manual_county,jurisdiction,pda_id,...,alt_zoning,zonetype,geom_id,Unnamed: 0,zoning_id,zoning,juris,prop,tablename,nodev
0,229116.0,0.0,3.360520,1.0,0.0,099 029001700,10305106092872,1,41992,NaN,...,NaN,NaN,1.030511e+13,1771381.0,60126.0,107 - Urban Low Residential UL2,-9999.0,100.0,plu06,0.0
1,244166.0,0.0,1.294423,1.0,0.0,099B540210200,11107351665227,1,41992,NaN,...,NaN,NaN,1.110735e+13,1362648.0,11903.0,GP-ULM,99.0,100.0,livermoregeneralplan,0.0
2,202378.0,6036500.0,14.993605,1.0,1.0,085A643106000,11030175960628,1,33000,NaN,...,NaN,NaN,1.103018e+13,307258.0,11803.0,LDR,98.0,100.0,hayward_gp_landuse,0.0
3,2004420.0,179954.0,316.247146,97.0,0.0,141-100-012,6381677629073,97,97,NaN,...,NaN,NaN,6.381678e+12,1737625.0,12975.0,LEA240,109.0,100.0,sonomacountygeneralplan,0.0
4,340332.0,0.0,0.621275,1.0,0.0,525 166004800,314875459798,1,26000,NaN,...,TE,TPA,3.148755e+11,273989.0,2511.0,RESM4,5.0,100.0,fremontgeneralplan,0.0


In [353]:
# Number of parcels missing zoning designation
z10_missing = p10_z10.loc[p10_z10['geom_id'].isnull()]
print(z10_missing.shape[0])
print(z10_missing.shape[0]/pz10.shape[0])


5476
0.002807149927745109


In [379]:
list(p10_z10)

['PARCEL_ID',
 'LAND_VALUE',
 'ACRES',
 'COUNTY_ID',
 'TAX_EXEMPT',
 'APN',
 'geom_id_s',
 'manual_county',
 'jurisdiction',
 'pda_id',
 'tpp_id',
 'exp_id',
 'exp_score',
 'opp_id',
 'zoningmodcat',
 'perffoot',
 'perfarea',
 'mapshell',
 'tpa_id',
 'perfarea2',
 'alt_zoning',
 'zonetype',
 'geom_id',
 'Unnamed: 0',
 'zoning_id',
 'zoning',
 'juris',
 'prop',
 'tablename',
 'nodev']

In [380]:
p10_z10.loc[p10_z10['jurisdiction'] != p10_z10['juris']]

,PARCEL_ID,LAND_VALUE,ACRES,COUNTY_ID,TAX_EXEMPT,APN,geom_id_s,manual_county,jurisdiction,pda_id,...,alt_zoning,zonetype,geom_id,Unnamed: 0,zoning_id,zoning,juris,prop,tablename,nodev
0,229116.0,0.0,3.360520,1.0,0.0,099 029001700,10305106092872,1,41992,NaN,...,NaN,NaN,1.030511e+13,1771381.0,60126,107 - Urban Low Residential UL2,-9999.0,100.0,plu06,0.0
1,244166.0,0.0,1.294423,1.0,0.0,099B540210200,11107351665227,1,41992,NaN,...,NaN,NaN,1.110735e+13,1362648.0,11903,GP-ULM,99.0,100.0,livermoregeneralplan,0.0
2,202378.0,6036500.0,14.993605,1.0,1.0,085A643106000,11030175960628,1,33000,NaN,...,NaN,NaN,1.103018e+13,307258.0,11803,LDR,98.0,100.0,hayward_gp_landuse,0.0
3,2004420.0,179954.0,316.247146,97.0,0.0,141-100-012,6381677629073,97,97,NaN,...,NaN,NaN,6.381678e+12,1737625.0,12975,LEA240,109.0,100.0,sonomacountygeneralplan,0.0
4,340332.0,0.0,0.621275,1.0,0.0,525 166004800,314875459798,1,26000,NaN,...,TE,TPA,3.148755e+11,273989.0,2511,RESM4,5.0,100.0,fremontgeneralplan,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956203,580747.0,29321.0,0.071424,13.0,0.0,271293,17158666132196,13,2252,NaN,...,NaN,NaN,1.715867e+13,47206.0,10204,MDR,82.0,100.0,antiochgeneralplan,0.0
1956204,594355.0,30019.0,0.137534,13.0,0.0,283543,16389503450045,13,16000,NaN,...,NaN,NaN,1.638950e+13,1526552.0,10702,LDR,87.0,100.0,concordparcels_gp120607_clip,0.0
1956205,804156.0,0.0,0.019658,41.0,0.0,14049007,1496694834659,41,52582,NaN,...,NaN,NaN,1.496695e+12,470288.0,8213,306 - Business and Professional Office,62.0,100.0,novatogp2006db,0.0
1956206,646401.0,517899.0,0.254764,13.0,0.0,33040,10694584892329,13,13,NaN,...,NaN,NaN,1.069458e+13,1695524.0,12302,SL,103.0,100.0,cccountygplanduse,0.0


In [275]:
#p10_z10.to_csv('p10_z10.csv')

# P10 with PBA40 PLU

In [354]:
p10_z10.loc[p10_z10['zoning_id'].isnull()] = 0
p10_z10['zoning_id'] = p10_z10['zoning_id'].apply(lambda x: int(x)) 
p10_z10.shape

(1956208, 30)

In [355]:
p10_plu10 = p10_z10.merge(plu10, left_on = 'zoning_id', right_on = 'id', how = 'left')

In [356]:
p10_plu10.shape

(1956208, 54)

In [357]:
p10_plu10.head()

,PARCEL_ID,LAND_VALUE,ACRES,COUNTY_ID,TAX_EXEMPT,APN,geom_id_s,manual_county,jurisdiction,pda_id,...,IL,IW,IH,RS,RB,MR,MT,ME,plandate,jz_o
0,229116.0,0.0,3.360520,1.0,0.0,099 029001700,10305106092872,1,41992,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Livermore 107 - Urban Low Residential UL2
1,244166.0,0.0,1.294423,1.0,0.0,099B540210200,11107351665227,1,41992,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Livermore GP-ULM
2,202378.0,6036500.0,14.993605,1.0,1.0,085A643106000,11030175960628,1,33000,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Hayward LDR
3,2004420.0,179954.0,316.247146,97.0,0.0,141-100-012,6381677629073,97,97,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Unincorporated Sonoma LEA240
4,340332.0,0.0,0.621275,1.0,0.0,525 166004800,314875459798,1,26000,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Fremont RESM4


In [359]:
list(p10_plu10)

['PARCEL_ID',
 'LAND_VALUE',
 'ACRES',
 'COUNTY_ID',
 'TAX_EXEMPT',
 'APN',
 'geom_id_s',
 'manual_county',
 'jurisdiction',
 'pda_id',
 'tpp_id',
 'exp_id',
 'exp_score',
 'opp_id',
 'zoningmodcat',
 'perffoot',
 'perfarea',
 'mapshell',
 'tpa_id',
 'perfarea2',
 'alt_zoning',
 'zonetype',
 'geom_id',
 'Unnamed: 0',
 'zoning_id',
 'zoning',
 'juris_x',
 'prop',
 'tablename',
 'nodev',
 'id',
 'juris_y',
 'city',
 'name',
 'max_far',
 'max_height',
 'max_dua',
 'max_du_per_parcel',
 'HS',
 'HT',
 'HM',
 'OF',
 'HO',
 'SC',
 'IL',
 'IW',
 'IH',
 'RS',
 'RB',
 'MR',
 'MT',
 'ME',
 'plandate',
 'jz_o']

## PLU Statistics

In [389]:
# Number of Units by jurisdiction
p10_plu10_main = p10_plu10[['COUNTY_ID','city','jurisdiction','juris_x','juris_y','ACRES','geom_id_s','max_far','max_dua','jz_o']]
p10_plu10_main['units'] = p10_plu10_main['ACRES'] * p10_plu10_main['max_dua']
p10_plu10_main['sf'] = p10_plu10_main['ACRES'] * p10_plu10_main['max_far'] * 43560


C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [390]:
p10_plu10_main

,COUNTY_ID,city,jurisdiction,juris_x,juris_y,ACRES,geom_id_s,max_far,max_dua,jz_o,units,sf
0,1.0,Livermore,41992,-9999.0,NaN,3.360520,10305106092872,NaN,2.00000,Livermore 107 - Urban Low Residential UL2,6.721041,NaN
1,1.0,Livermore,41992,99.0,99.0,1.294423,11107351665227,NaN,3.00000,Livermore GP-ULM,3.883268,NaN
2,1.0,Hayward,33000,98.0,98.0,14.993605,11030175960628,NaN,8.70000,Hayward LDR,130.444362,NaN
3,97.0,Unincorporated Sonoma,97,109.0,109.0,316.247146,6381677629073,NaN,0.00417,Unincorporated Sonoma LEA240,1.318751,NaN
4,1.0,Fremont,26000,5.0,5.0,0.621275,314875459798,NaN,23.00000,Fremont RESM4,14.289334,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1956203,13.0,Antioch,2252,82.0,82.0,0.071424,17158666132196,NaN,10.00000,Antioch MDR,0.714239,NaN
1956204,13.0,Concord,16000,87.0,87.0,0.137534,16389503450045,NaN,10.00000,Concord LDR,1.375336,NaN
1956205,41.0,Novato,52582,62.0,62.0,0.019658,1496694834659,0.4,NaN,Novato 306 - Business and Professional Office,NaN,342.523479
1956206,13.0,Unincorporated Contra Costa,13,103.0,103.0,0.254764,10694584892329,NaN,2.90000,Unincorporated Contra Costa SL,0.738815,NaN


In [404]:
boc_j_10 = p10_plu10_main.groupby(['city'])['ACRES','units','sf'].sum().reset_index()
boc_j_10['ACRES'] = boc_j_10['ACRES'].apply(lambda x: f'{int(x):,}')
boc_j_10['units'] = boc_j_10['units'].apply(lambda x: f'{int(x):,}')
boc_j_10['sf'] = boc_j_10['sf'].apply(lambda x: f'{int(x):,}')

In [405]:
display(boc_j_10)

,city,ACRES,units,sf
0,Alameda,"3,049","34,091","59,003,192"
1,Albany,737,"10,033","18,121,816"
2,American Canyon,"1,769","6,393","7,337,397"
3,Antioch,"12,073","88,487","174,355,891"
4,Atherton,"2,735","2,526","20,873,282"
...,...,...,...,...
112,Vallejo,"11,911","67,142","114,573,344"
113,Walnut Creek,"8,552","51,996","54,422,627"
114,Windsor,"3,487","15,462","9,601,450"
115,Woodside,"6,955","2,163","312,021"


In [412]:
boc_cty_10 = p10_plu10_main.groupby(['COUNTY_ID'])['ACRES','units','sf'].sum().reset_index()
boc_cty_10 = boc_cty_10.loc[boc_cty_10['COUNTY_ID'] > 0]
boc_cty_10['ACRES'] = boc_cty_10['ACRES'].apply(lambda x: f'{int(x):,}')
boc_cty_10['units'] = boc_cty_10['units'].apply(lambda x: f'{int(x):,}')
boc_cty_10['sf'] = boc_cty_10['sf'].apply(lambda x: f'{int(x):,}')
boc_c_10 = boc_cty_10.merge(ctyMap, left_on = 'COUNTY_ID', right_on = 'ctyCode', how = 'left')
boc_c_10.drop(columns = ['ctyCode'],inplace=True)
boc_c_10

,COUNTY_ID,ACRES,units,sf,ctyName
0,1.0,"479,221","1,132,680","2,360,201,919",Alameda
1,13.0,"445,742","768,422","1,059,290,704",Contra Costa
2,41.0,"371,379","223,537","191,450,982",Marin
3,55.0,"495,736","464,635","778,118,312",Napa
4,75.0,"22,682","411,420","1,405,478,871",San Francisco
5,81.0,"291,369","394,233","979,620,948",San Mateo
6,85.0,"791,981","1,028,659","7,985,213,594",Santa Clara
7,95.0,"528,653","239,209","690,367,399",Solano
8,97.0,"992,967","299,285","1,679,114,336",Sonoma


## Join zoning Scenarios to Parcel10

In [363]:
zmods = pd.read_csv('C:/Users/ywang/Box/Mine/1_UrbanSim/03_02_2020_parcels_geography.csv')
zmods.head()

,Unnamed: 0,geom_id,jurisdiction_id,pda_id,tpp_id,exp_id,opp_id,zoningmodcat,perffoot,perfarea,urbanized,juris_id,hra_id,trich_id,cat_id,zoninghzcat,pba50zoningmodcat
0,0,10305106092872,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0,1,livr,HRA,NaN,NaN,livrHRANANA,livrNANAHRADRNAinNA
1,1,11107351665227,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0,1,livr,NaN,NaN,NaN,livrNANANA,livrNANADRNAinNA
2,2,11030175960628,33000,NaN,NaN,NaN,NaN,33000NANANANA,1,0,0,hayw,NaN,NaN,NaN,haywNANANA,haywNANANANAinNA
3,3,6381677629073,97,NaN,NaN,NaN,NaN,00097NANANANA,0,0,0,uson,NaN,NaN,NaN,usonNANANA,usonNANADRNAoutNA
4,4,314875459798,26000,NaN,b1,NaN,NaN,26000NAb1NANA,1,1,1,frem,HRA,Conn,NaN,fremHRAConnNA,fremNANAHRADRNAinNA


In [364]:
zmod_ls = list(zmods)

In [365]:
p10_plu10_zmods = p10_plu10.merge(zmods, left_on = 'geom_id_s', right_on = 'geom_id', how = 'left')
p10_plu10_zmods.shape

(1956208, 71)

In [366]:
p10_plu10_zmods.head()

,PARCEL_ID,LAND_VALUE,ACRES,COUNTY_ID,TAX_EXEMPT,APN,geom_id_s,manual_county,jurisdiction,pda_id_x,...,zoningmodcat_y,perffoot_y,perfarea_y,urbanized,juris_id,hra_id,trich_id,cat_id,zoninghzcat,pba50zoningmodcat
0,229116.0,0.0,3.360520,1.0,0.0,099 029001700,10305106092872,1,41992,NaN,...,41992NANANANA,1.0,0.0,1.0,livr,HRA,NaN,NaN,livrHRANANA,livrNANAHRADRNAinNA
1,244166.0,0.0,1.294423,1.0,0.0,099B540210200,11107351665227,1,41992,NaN,...,41992NANANANA,1.0,0.0,1.0,livr,NaN,NaN,NaN,livrNANANA,livrNANADRNAinNA
2,202378.0,6036500.0,14.993605,1.0,1.0,085A643106000,11030175960628,1,33000,NaN,...,33000NANANANA,1.0,0.0,0.0,hayw,NaN,NaN,NaN,haywNANANA,haywNANANANAinNA
3,2004420.0,179954.0,316.247146,97.0,0.0,141-100-012,6381677629073,97,97,NaN,...,00097NANANANA,0.0,0.0,0.0,uson,NaN,NaN,NaN,usonNANANA,usonNANADRNAoutNA
4,340332.0,0.0,0.621275,1.0,0.0,525 166004800,314875459798,1,26000,NaN,...,26000NAb1NANA,1.0,1.0,1.0,frem,HRA,Conn,NaN,fremHRAConnNA,fremNANAHRADRNAinNA


In [367]:
p10_plu10_zmods.loc[p10_plu10_zmods['jurisdiction'] != p10_plu10_zmods['juris_x']][['jurisdiction','juris_x','juris_id']]

,jurisdiction,juris_x,juris_id
0,41992,-9999.0,livr
1,41992,99.0,livr
2,33000,98.0,hayw
3,97,109.0,uson
4,26000,5.0,frem
...,...,...,...
1956203,2252,82.0,anti
1956204,16000,87.0,conc
1956205,52582,62.0,nova
1956206,13,103.0,ucnc


## Zoning Scenario Statistics

In [377]:
p10_plu10_zmods.groupby(['juris_id','pba50zoningmodcat'])['PARCEL_ID'].count()

juris_id  pba50zoningmodcat   
alam      alamGGNAHRADRNAinNA     1551
          alamGGNAHRADRNAinsfd     710
          alamGGNAHRANAinNA       1561
          alamGGNAHRANAinsfd      1973
          alamGGNANANAinNA         284
                                  ... 
wood      woodNANANANAoutsfd         1
youn      unapNANADRNAoutNA          2
          younNANADRNAinNA         452
          younNANADRNAinsfd        383
          younNANADRNAoutNA          3
Name: PARCEL_ID, Length: 2650, dtype: int64

In [378]:
p10_plu10_zmods.groupby(['juris_id','pba50zoningmodcat'])['ACRES'].sum()

juris_id  pba50zoningmodcat   
alam      alamGGNAHRADRNAinNA     268.521719
          alamGGNAHRADRNAinsfd     83.154525
          alamGGNAHRANAinNA       350.678271
          alamGGNAHRANAinsfd      206.566050
          alamGGNANANAinNA         51.338104
                                     ...    
wood      woodNANANANAoutsfd        1.482699
youn      unapNANADRNAoutNA        79.914943
          younNANADRNAinNA        617.652537
          younNANADRNAinsfd        92.807608
          younNANADRNAoutNA        43.340640
Name: ACRES, Length: 2650, dtype: float64